In [1]:
!pip install PyPDF2

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install textract

  Using cached https://files.pythonhosted.org/packages/32/31/ef9451e6e48a1a57e337c5f20d4ef58c1a13d91560d2574c738b1320bb8d/textract-1.6.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4d/82/f44c9661e479207348a979b1f6f063625d11dc4ca6256af053719bbb0124/argcomplete-1.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/fd/6e8746e6965d1a7ea8e97253e3d79e625da5547e8f376f88de5d024bacb9/pdfminer.six-20181108-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/16/63576a1a001752e34bf8ea62e367997530dc553b689356b9879339cf45a4/xlrd-1.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/26/e1/7f5678cd94ec1234269d23756dbdaa4c8cfaed973412f88ae8adf7893a50/SpeechRecognition-3.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.or

In [9]:
import textract
text = textract.process("WVU2018-2019.pdf")

In [10]:
text.decode('utf-8').replace('\n', '').split('\x0c')[0]

'West Virginia University      1Undergraduate Course DescriptionsACCOUNTING COURSESACCT 191. First-Year Seminar. 1-3 Hours.Engages students in active learning strategies that enable effective transition to college life at WVU. Students will explore school, college and universityprograms, policies and services relevant to academic success. Provides active learning activities that enable effective transition to the academicenvironment. Students examine school, college and university programs, policies and services.ACCT 201. Principles of Accounting. 3 Hours.The concepts, principles, and procedures pertaining to the preparation, analysis, and interpretation of financial statements.ACCT 202. Principles of Accounting. 3 Hours.PR: ACCT 201 with grade of C or better. Utilization of accounting information for purposes of managerial control and decision making; cost concepts,profit and financial budgeting, analysis of financial statements.ACCT 293. Special Topics. 1-6 Hours.PR: Consent. Investi

In [5]:
import PyPDF2 as pypdf
import regex as re
import numpy as np
import csv
import pandas as pd

In [14]:
PDFfile = open("2018-2019-Kentucky-State-University-Catalogue.pdf", "rb")
pdfread = pypdf.PdfFileReader(PDFfile)
pdfread

In [19]:
y = pdfread.getPage(164)
y.extractText().replace('\n', '')

'156  Course Descriptions  AFRICAN-AMERICAN STUDIES  AAS 303/ART 303: African-American Art History. A course designed to reveal contributions and accomplishments of African-American artists and the influence of traditional early African art on art of the world. Credit: 3 semester hours.  AAS 314/HIS 314: African-Americans in the United States. The history of African-Americans in the United States, their role in the development of this country, and their struggle for rights, privileges, and protections of citizenship. Prerequisite(s): HIS 201 and HIS 202, or consent of instructor. Credit: 3 semester hours.  AAS 323/MUS 323: African-American Music. A study of the development of African-American music, including spirituals, blues, jazz, symphonic literature, and miscellaneous songs. Open to non-Music majors. Credit: 3 semester hours  AAS 325/MUS 325: Jazz History and Literature. A survey of jazz, including its origins, major performers, and composers. All idioms and styles are studied usi

In [5]:
pdfread.getNumPages()

285

In [20]:
def get_data(csv_name, pdf_name, page_range, pattern, dep_group, abbrev_group, c_group, t_group, desc_group,
            year, grad_status):
    """Takes in name of PDF in local folder and extracts data in given PAGE_RANGE 
    using regex expression PATTERN and given group numbers. YEAR is the catalog year.
    GRAD_STATUS is either None (if unknown), 'G', 'UG', or a number indicating at
    which number is the course graduate."""
    
    PDFfile = open(pdf_name, "rb")
    pdfread = pypdf.PdfFileReader(PDFfile)
    
    csv_writer = csv.writer(open(csv_name, 'w'))
    csv_writer.writerow(["Department Name", "Course Catalogue Number", "Course Name",
     "Course Description", "Graduate/Undergraduate", "Format", "Lab", 
     "Academic Catalogue Year"])
    FS_keys = ["agri", "food", "animal"]
    
    look_ahead = pattern + r'(.+?)(?=' + pattern + r')'
    
    dep_names = {}
    #test = page_range[0] #DEBUGGING
    text = ''
    for i in np.arange(page_range[0], page_range[1]):
    #    print(test) #DEBUGGING
    #    test += 1 #DEBUGGING
        text += pdfread.getPage(i).extractText().replace('\n', ' ')    
    matches = re.findall(look_ahead, text)
    last = re.sub(look_ahead, '', text)
    last = re.search(pattern + r'(.+)', last)
    matches = matches + [last.groups()]

    for match in matches:  
        #print(match) #DEBUGGING
        # Getting full department name
        print(dep_names)
        if not dep_names.get(match[abbrev_group - 1]):
            dep_names[match[abbrev_group - 1]] = match[dep_group - 1]
        dep_name = dep_names.get(match[abbrev_group - 1])

        # Other groups
        course_num = int(match[c_group - 1])
        title = str(match[t_group - 1])
        desc = str(match[desc_group - 1]) # assuming description is group 5

        #Filtering
        #for key in FS_keys:
        #    if (re.search(key, title, flags=re.IGNORECASE) or 
        #        re.search(key, desc, flags=re.IGNORECASE)):
        G_UG = grad_status
        if grad_status:
            if type(grad_status) == int:
                G_UG = "UG"
                if int(t.group(1)) >= grad_status:
                    G_UG = "G"
        lab = False
        if (re.search(r'Laboratory|Lab', title, flags=re.IGNORECASE) or 
        re.search(r'Laboratory|Lab', desc, flags=re.IGNORECASE)):
            lab = True
        csv_writer.writerow([dep_name, course_num, title, desc, 
                                 G_UG, None, lab, year])
                                # Leaving format blank for now
                        
    print(dep_names)
                
pattern = r'([A-Z\s?]+)?\s{2}?([A-Z]{3,4})\s+(\d{3,4})\/?(.+?):?(.+?)\.'
#Partial Test set:
get_data('KentuckyStateU.csv', "2018-2019-Kentucky-State-University-Catalogue.pdf", [164, 253], pattern, 
         1, 2, 3, 5, 6, '2018-2019', None)

{}
{'AAS': ' AMERICAN STUDIES   '}
{'AAS': ' AMERICAN STUDIES   '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' ', 'ACC': '     ACCOUNTING   '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' ', 'ACC': '     ACCOUNTING   ', 'MAT': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' ', 'ACC': '     ACCOUNTING   ', 'MAT': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' ', 'ACC': '     ACCOUNTING   ', 'MAT': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' ', 'ACC': '     ACCOUNTING   ', 'MAT': ' '}
{'AAS': ' AMERICAN STUDIES   ', 'HIS': ' ', 'ACC': '     ACCOUNTING   ', 'MAT': 

In [21]:
pd.read_csv('KentuckyStateU.csv')

,Department Name,Course Catalogue Number,Course Name,Course Description,Graduate/Undergraduate,Format,Lab,Academic Catalogue Year
0,AMERICAN STUDIES,303,RT 303: African - American Art History,A course designed to reveal contributions an...,NaN,NaN,False,2018-2019
1,AMERICAN STUDIES,314,IS 314: African - Americans in the United States,The history of African - Americans in the Uni...,NaN,NaN,False,2018-2019
2,,201,"and HIS 202, or consent of instructor",Credit: 3 semester hours.,NaN,NaN,False,2018-2019
3,AMERICAN STUDIES,323,US 323: African - American Music,A s tudy of the development of African - Ame...,NaN,NaN,False,2018-2019
4,AMERICAN STUDIES,325,US 325: Jazz History and Literature,"A survey of jazz, includi ng its origins, ma...",NaN,NaN,False,2018-2019
5,AMERICAN STUDIES,365,IS 365: African History to 1870,A survey tracing the earliest beginnin...,NaN,NaN,False,2018-2019
6,,101,and HIS 102,Credit: 3 semester hours.,NaN,NaN,False,2018-2019
7,AMERICAN STUDIES,366,IS 366: African History from 1870,The creation and development of European col...,NaN,NaN,False,2018-2019
8,,101,and HIS 102,Credit: 3 semester hours.,NaN,NaN,False,2018-2019
9,AMERICAN STUDIES,409,NG 409: Literature of African - Americans,A study of representative literary w...,NaN,NaN,False,2018-2019


In [13]:
sample = '156  Course Descriptions  AFRICAN-AMERICAN STUDIES  AAS 303/ART 303: African-American Art History. A course designed to reveal contributions and accomplishments of African-American artists and the influence of traditional early African art on art of the world. Credit: 3 semester hours.  AAS 314/HIS 314: African-Americans in the United States. The history of African-Americans in the United States, their role in the development of this country, and their struggle for rights, privileges, and protections of citizenship. Prerequisite(s): HIS 201 and HIS 202, or consent of instructor. Credit: 3 semester hours.  AAS 323/MUS 323: African-American Music. A study of the development of African-American music, including spirituals, blues, jazz, symphonic literature, and miscellaneous songs. Open to non-Music majors. Credit: 3 semester hours  AAS 325/MUS 325: Jazz History and Literature. A survey of jazz, including its origins, major performers, and composers. All idioms and styles are studied using selected readings and recordings. Open to non-Music majors. Credit: 3 semester hours.  AAS 365/HIS 365: African History to 1870. A survey tracing the earliest beginnings of man in Africa and the development of sub-and trans-Saharan civilizations. Prerequisites: HIS 101 and HIS 102. Credit: 3 semester hours.  AAS 366/HIS 366: African History from 1870. The creation and development of European colonial empires in Africa and the subsequent rise of independent African states. Prerequisites: HIS 101 and HIS 102. Credit: 3 semester hours.  AAS 409/ENG 409: Literature of African-Americans. A study of representative literary works written by African Americans from    Phillis Wheatley   to the present. Prerequisite: ENG 211. Credit: 3 semester hours.  AAS 411/HIS 411: African-Americans in the Twentieth Century. A thorough study of the contributions of African Americans to American culture and their struggle for full citizenship in the United States. Prerequisites: HIS 201 and HIS 202. Credit: 3 semester hours.  ACCOUNTING  ACC 201: Principles of Accounting I. An introduction to current accounting theory and practice, including accounting concepts, principles, financial statements, the accounting cycle, and selected assets such as cash, receivables, inventory and long-term assets. Prerequisites: MAT 115 or higher, BUA 101 and BUA 102 with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 202: Principles of Accounting II. A continuation of ACC 201. A study of accounting fundamentals including current liabilities, partnerships, corporations, the statement of cash flows, investments and financial performance evaluation, and accounting information for managerial decision-making. Prerequisites: ACC 201, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 300: Managerial Accounting. An exploration of decision-making tools and use of accounting information used by internal management for directing Prerequisites: ACC 201, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 301: Intermediate Accounting I. A comprehensive study of concepts and principles underlying corporate financial accounting and reporting. Emphasis is given to current assets and operating assets. Prerequisites: ACC 202, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 302: Intermediate Accounting II. A study of investments, current and long-term liabilities, changes and error analysis, statement of cash flows, and interperiod income tax allocation. Prerequisite: ACC 301. A continuation of ACC 301. Credit: 3 semester hours.  ACC 303: Intermediate Accounting III. This course is a continuation of ACC 301 and 302. The course consists of a comprehensive study of concepts and principles underlying current financial accounting and reporting practices, including IFRS. Prerequisites: ACC 302 with grade of C or better. Topics include pensions, leases, accounting changes and error analysis, accounting for income taxes, statement of cash flows, governmental and non-profit accounting, business combinations and consolidated financial statements. Credit: 3 semester hours.  ACC 308: Cost Accounting I. The study of cost behavior from a managerial perspective. The principles, practices, and procedures involved in collecting, assembling, analyzing, and controlling manufacturing costs are examined. Prerequisites: ACC 202, BUA 101 and BUA 102, all with grades of C or better or consent of instructor. Credit: 3 semester hours.  ACC 324: Accounting Information Systems. An exploration of the flow of accounting information and '
test = '([A-Z\s?]+)?\s{2}([A-Z]{3,4})\s+(\d{3,4})\/?(.+?):?(.+?)\.'
rest = re.sub(test + r'(.+?)(?=' + test + r')', '',sample)
rest

'156  Course Descriptions  AFRICAN-  ACC 324: Accounting Information Systems. An exploration of the flow of accounting information and '

In [18]:
rest_match = re.search(cmn_pattern + r'(.+)', rest)
print(rest_match.group(2))
print(rest_match.group(3))
print(rest_match.group(4))
print(rest_match.group(5))
print(rest_match.group(7))
rest_match.groups()

None
HCA
2533
Utilization of Community Resources
Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.


(None,
 None,
 'HCA',
 '2533',
 'Utilization of Community Resources',
 ' be',
 'Students will be taught the value of fostering good community relations that directly impact program services. The function of various community services will be assessed as well as the study of those resources to enhance the quality of assistance to clients.')

In [15]:
sample = '20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION   ACCOUNTING (ACC)   ACC 100 Fundamentals of Accounting (3)   Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a ccounting; fund accounts; and nonprofit accounting for  entities such as public colleges, universities and health care providers .   ACC 350 Finance (3)    ACC 310,  BUS 265, ECO 200 or 201.   Introduction to the theories and applications associated with the functions of capital markets. This  includes the conceptual foundations of portfolio theory, risk management, and asset valuation. This  course will examine the markets for equity, money, and  bonds. One hour laboratory required.   AGRICULTURE (AGR)   AGR 103 Conservation Planning (4)   This course introduces students to the understanding natural resource conservation issues and concerns  to allow them to develop conservation plans necessary to improve   natural resource conditions applicable  to tribal lands.   AGR 105 Int roduction to Animal Science (4)   This course will introduce students to animal and livestock industries, production, systems and markets.  Students will not only learn about national product ion but include Navajo Nation livestock industry.   AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '
dine_pattern = r'(\w+\s\(\w{2,3}\))?\s+([a-zA-Z]{2,3})\s([\d]+\w?)\s([^\d]+)\(\d\)\s+(.+?)'
last = re.sub(dine_pattern + '(?=' + dine_pattern + ')', '', sample)
l

'20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION      AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '

In [9]:
r'test' + r'test2'

'testtest2'

In [15]:
dine_pattern + r'(?=' + dine_pattern + r')'

'(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?)(?=(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?))'

In [33]:
sample
re.findall(dine_pattern + r'(.+)(?=' + dine_pattern + r')', sample)[0]

('ACCOUNTING (ACC)',
 'ACC',
 '100',
 'Fundamentals of Accounting ',
 'Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a

In [65]:
if not None:
    print(True)

True
